# Imports

Install sleap

In [ ]:
! pip install sleap

In [ ]:
import os
import numpy as np
import sleap

# Load the Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Download the `cropped_video1080.zip ` from Google Drive (https://drive.google.com/drive/folders/1xzN19rTxewEiwzeBN3g88w5UBN2Z0Z0S?usp=share_link) and add it to your Drive or local directory if you are not using Google Colab.

Next step is to unzip the file.

In [ ]:
! unzip "cropped_video1080.zip"

# Utility Functions/Classes
To obtain the prediction in a numpy format that we can use to train and/or our models we need to obtain the slp (i.e., sleap) predictions from the video using the model trained using SLEAP. Then with those predictions we can obtain the numpies.

## From videos to SLP
From a mp4 video file it obtaines the slp prediction.

In the code the following functions is located in `poseestimation.predictionheandling`.

In [ ]:
def get_predictions(video_path: str, model_path: str, pred_output: str, with_images: bool):
    """
    Gets the prediction from a video using a specific trained model, and saves it as a slp file
    :param with_images: boolean indicating if the predictions must contain also the images
    :param video_path: string indicating the path to the video
    :param model_path: string indicating the path to the model
    :param pred_output: string indicating where the predictions are saved
    :return:
    """
    sleap.disable_preallocation()

    video = sleap.load_video(video_path)
    print(video.shape)

    predictor = sleap.load_model(model_path)
    predictions = predictor.predict(video)

    predictions = sleap.Labels(predictions.labeled_frames[:])
    predictions.save(pred_output, with_images=with_images, embed_all_labeled=True)

## From SLP to numpy
From a slp file it obtaines the corresponding numpy in the format (10, frames), where 10 is the number of joints times 2, i.e., the coordinates x and y. In the order the joints are:


*   Head
*   Right beginning of the antenna
* Right tip of the antenna
*   Left beginning of the antenna
* Left tip of the antenna


In the code this following function is located in `posestimation.predictionhandling`

In [ ]:
def obtain_np_seq_from_slp_pred(predictions_path, output_path: str):
    """
    Save in a npy file the predictions from a slp file
    :param predictions_path: path to where the slp file is located
    :param output_path: path where to save the npy file containing the predictions
    :return:
    """
    predictions = sleap.load_file(predictions_path)
    dataset = []
    for label in predictions:
        frame_pred = []
        for values in label[0].numpy():
            frame_pred.append(values[0])
            frame_pred.append(values[1])
        dataset.append(frame_pred)

    np.save(output_path, dataset)

# Perform Prediction and SLP to npy transformation

Here you need to set the path to the appropriate locations and to tell the code if you want to save also the images when predicting each video frame. If you do that set `save_slp_with_images=True`, in this way you can open the slp prediction in your SLEAP GUI and analyze how the results are. Be aware that this operation costs a lot of additional memory, thus we recommend to set the value to `True` only if you want to see the results on SLEAP otherwise is better to have it at `False` (i.e., only the predictions without the images/frames will be saves in the slp files).

In [ ]:
video_path = 'videos 2 minutes'
prediction_slp_path = 'predictions_slp' # directory to where the slp files will be saved
save_slp_with_images = True
prediction_npy_path = 'predictions_npy' # directory to where the npy files will be saved

If the directories referred by prediction_slp_path and prediction_npy_path do not exist, there is the need to create them. If they exists the following snippet will do nothing, so you can skip it.

In [ ]:
if not os.path.exists(prediction_slp_path):
    os.mkdir(prediction_slp_path)
if not os.path.exists(prediction_npy_path):
    os.mkdir(prediction_npy_path)

Now we need to navigate the directories performing first the prediction and then the transformation to numpy.

In [ ]:
classes = os.listdir(video_path)
for c in classes: # control sugar ammonia ..
    if c not in ['control', 'sugar', 'ammonia']:
        continue
    class_path = os.path.join(video_path, c) # e.g., videos/control/
    sets = os.listdir(class_path)

    prediction_class_slp_path = os.path.join(prediction_slp_path, c)    # e.g., predictions_slp/control
    os.mkdir(prediction_class_slp_path)

    prediction_class_npy_path = os.path.join(prediction_npy_path, c)  # e.g., predictions_npy/control
    os.mkdir(prediction_class_npy_path)

    for s in sets: # train # test # val
        if s not in ['train', 'test', 'val']:
            continue

        set_path = os.path.join(class_path, s)  # e.g., video/control/train
        videos = [video for video in os.listdir(set_path) if video.endswith('.mp4')]

        prediction_set_slp_path = os.path.join(prediction_class_slp_path, s)
        os.mkdir(prediction_set_slp_path)

        prediction_set_npy_path = os.path.join(prediction_class_npy_path, s)
        os.mkdir(prediction_set_npy_path)

        for video in videos:
            get_predictions(
                os.path.join(set_path, video),
                'm64_64_1.0/221226_195308.single_instance',
                os.path.join(prediction_set_slp_path, video[:-4] + '.pkg.slp'),
                save_slp_with_images
            )

            obtain_np_seq_from_slp_pred(
                os.path.join(prediction_set_slp_path, video[:-4] + '.pkg.slp'),
                os.path.join(prediction_set_npy_path, video[:-4] + '.npy')
            )